In [ ]:
using Random
using Statistics

using AbstractPlotting
using CairoMakie
using LightGraphs
using JuMP
using Parameters
using SCIP
using Triangle

In [ ]:
# create instance data
Random.seed!(0);

const N = 7
const WIDTH = 500
const HEIGHT = 300

x = 0.95 * WIDTH * rand(N)
y = 0.95 * HEIGHT * rand(N)

points = [x y]

In [ ]:
function make_scene(width=WIDTH, height=HEIGHT)
    return Scene(resolution=(width, height), show_axis=false, scale_plot=false)
end

In [ ]:
function draw_points!(points; markersize=4, color=:black)
    scatter!(points, markersize=markersize, color=color)
end

In [ ]:
make_scene()
draw_points!(points)

In [ ]:
struct Triangulation
    points::Matrix{Float64}   # n x 2
    edges::Matrix{Int64}      # m x 2
    triangles::Matrix{Int64}  # t x 3
end

In [ ]:
function unique_edges(triangles)
    set = Set()
    for t in 1:size(triangles, 1)
        triangle = triangles[t, :]
        push!(set, min(triangle[[1, 2]], triangle[[2, 1]]))
        push!(set, min(triangle[[2, 3]], triangle[[3, 2]]))
        push!(set, min(triangle[[1, 3]], triangle[[3, 1]]))
    end
    return hcat(sort(collect(set))...)'
end

In [ ]:
function delaunay_triangulation(points)::Triangulation
    points_map = collect(1:size(points, 1))
    triangle_array = Triangle.basic_triangulation(points, points_map)
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(points, edges, triangles)
end

In [ ]:
function draw_edges!(triangulation; color=:gray)
    @unpack points, edges = triangulation
    linesegments!(points[edges'[:], :], color=color)
end

In [ ]:
function draw_triangulation(triangulation)
    make_scene()
    draw_edges!(triangulation)
    draw_points!(triangulation.points)   
end

In [ ]:
del = delaunay_triangulation(points)
draw_triangulation(del)

In [ ]:
pointset_mean(array) = dropdims(mean(array, dims=2), dims=2)

In [ ]:
function triangle_centers(triangulation)
    @unpack points, triangles = triangulation
    return pointset_mean(points[triangles, :])
end

In [ ]:
centers = triangle_centers(del)
draw_points!(centers, color=:limegreen)

In [ ]:
function delaunay_with_centers(triangulation)
    centers = triangle_centers(triangulation)
    all_points = vcat(triangulation.points, centers)
    return delaunay_triangulation(all_points)
end

In [ ]:
draw_triangulation(delaunay_with_centers(del))

In [ ]:
draw_triangulation(delaunay_with_centers(delaunay_with_centers(del)))

In [ ]:
function constrained_with_centers(triangulation)
    @unpack points, edges = triangulation
    centers = triangle_centers(triangulation)
    all_points = vcat(points, centers)
    point_map = collect(1:size(all_points, 1))
    
    triangle_array = Triangle.constrained_triangulation(all_points, point_map, edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(all_points, edges, triangles)
end

In [ ]:
draw_triangulation(constrained_with_centers(del))

In [ ]:
draw_triangulation(constrained_with_centers(constrained_with_centers(del)))

In [ ]:
function edge_midpoints(triangulation)
    @unpack points, edges = triangulation
    return pointset_mean(points[edges, :])
end

In [ ]:
draw_triangulation(del)
draw_points!(edge_midpoints(del), color=:red)

In [ ]:
# triangulate with edge subdivision, again
del_ = delaunay_triangulation(vcat(del.points, edge_midpoints(del)))
draw_triangulation(delaunay_triangulation(vcat(del_.points, edge_midpoints(del_))))

In [ ]:
function subdivided_edges(edges, offset)
    set = Vector()
    for e in 1:size(edges, 1)
        edge = edges[e, :]
        push!(set, [edge[1] e + offset])
        push!(set, [e + offset edge[2]])
    end
    return vcat(set...)
end

In [ ]:
function constrained_subdivision(triangulation)
    @unpack points, edges = triangulation
    new_points = vcat(points, edge_midpoints(triangulation))
    point_map = collect(1:size(new_points, 1))
    keep_edges = subdivided_edges(edges, size(points, 1))

    triangle_array = Triangle.constrained_triangulation(new_points, point_map, keep_edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(new_points, edges, triangles)
end

In [ ]:
draw_triangulation(constrained_subdivision(del))

In [ ]:
draw_triangulation(constrained_subdivision(constrained_subdivision(del)))

In [ ]:
# alternate: first add triangle centers, then subdivide edges
draw_triangulation(constrained_subdivision(constrained_with_centers(del)))

In [ ]:
# alternate 2: first subdivide edges, the add triangle centers
draw_triangulation(constrained_with_centers(constrained_subdivision(del)))

In [ ]:
# alternate 2.5: first subdivide edges, the add triangle centers (unconstrained)
draw_triangulation(delaunay_with_centers(constrained_subdivision(del)))

In [ ]:
function constrained_combined_refinement(triangulation)
    @unpack points, edges = triangulation
    new_points = vcat(points, edge_midpoints(triangulation), triangle_centers(triangulation))
    point_map = collect(1:size(new_points, 1))
    keep_edges = subdivided_edges(edges, size(points, 1))

    triangle_array = Triangle.constrained_triangulation(new_points, point_map, keep_edges)
    
    triangles = hcat(triangle_array...)'
    edges = unique_edges(triangles)
    return Triangulation(new_points, edges, triangles)
end

In [ ]:
draw_triangulation(constrained_combined_refinement(del))

In [ ]:
draw_triangulation(constrained_combined_refinement(constrained_combined_refinement(del)))